In [1]:
#!/usr/bin/env python2
from __future__ import print_function
import os, sys, time, copy

import random
import numpy as np
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
from multiprocessing import freeze_support
import pints#!/usr/bin/env python
import bisect

sys.path.append('../')
import model_ikr as m
import parametertransform
from protocols import leak_staircase as protocol_def
from priors import prior_parameters
from priors_JK import BeattieLogPrior as LogPrior
from optimizatoinJK import OptimisationController

sys.path.append('../../../')
sys.path.append('../../../Protocols')
sys.path.append('../../../Models')
sys.path.append('../../../Lib')
import protocol_lib
import vc_protocols
from preprocessing_lib import get_modified_current, get_currents_with_constant_dt 


random.seed(542811797)
np.random.seed(542811797)
        
# Parameter range setting
dataset_dir = 'Kylie'
parameter_ranges = []
if 'Kylie' in dataset_dir:    
    if 'rmax600' in dataset_dir:
        # Kylie: rmax = 600 
        parameter_ranges = []
        parameter_ranges.append( [100, 500000] )
        parameter_ranges.append( [0.0001, 598] )
        parameter_ranges.append( [0.0001, 260] )
        parameter_ranges.append( [0.0001, 598] )
        parameter_ranges.append( [0.0001, 130] )
        parameter_ranges.append( [0.0001, 598] )
        parameter_ranges.append( [0.0001, 260] )
        parameter_ranges.append( [0.0001, 598] )
        parameter_ranges.append( [0.0001, 130] )
        print("Kylie-rmax600 dataset has been selected.")
    else :
        # Kylie
        parameter_ranges.append([100, 500000])
        parameter_ranges.append( [0.0001, 1000000])
        parameter_ranges.append( [0.0001, 384])
        parameter_ranges.append( [0.0001, 1000000] )
        parameter_ranges.append( [0.0001, 192] )
        parameter_ranges.append( [0.0001, 1000000] )
        parameter_ranges.append( [0.0001, 384] )
        parameter_ranges.append( [0.0001, 1000000] )
        parameter_ranges.append( [0.0001, 192] )
        print("Kylie dataset has been selected.")

elif 'RealRange' in dataset_dir:
        parameter_ranges.append([3134, 500000])                 # g
        parameter_ranges.append( [0.0001, 2.6152843264828003])  # p1
        parameter_ranges.append( [43.33271226094526, 259])      # p2
        parameter_ranges.append( [0.001, 0.5] )                 # p3
        parameter_ranges.append( [15, 75] )                     # p4
        parameter_ranges.append( [0.8, 410] )                   # p5
        parameter_ranges.append( [0.0001, 138.] )               # p6
        parameter_ranges.append( [1.0, 59] )                    # p7
        parameter_ranges.append( [1.6, 90] )                    # p8
        print("RealRange dataset has been selected.")

parameter_ranges = np.array(parameter_ranges)
print(parameter_ranges.shape)

 
param_names = ['g', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']


# Mean Squared Error
def MSE(y, y_pred, axis=1):
	return np.mean(np.square(y - y_pred), axis=axis)

C:\Anaconda3\envs\ml\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Anaconda3\envs\ml\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


Kylie dataset has been selected.
(9, 2)
Kylie dataset has been selected.
(9, 2)


In [2]:
a= np.ones(10)
b = np.zeros(10)
MSE(a,b, axis=0)

1.0

In [3]:
import cma
# help(cma)

# Set up simulation
options = cma.CMAOptions()

# Tell cma-es to be quiet
options.set('verbose', -9)

# Set population size
options.set('popsize', 4)

# CMAES always seeds np.random, whether you ask it too or not, so to
# get consistent debugging output, we should always pass in a seed.
# Instead of using a fixed number (which would be bad), we can use a
# randomly generated number: This will ensure pseudo-randomness, but
# produce consistent results if np.random has been seeded before
# calling.
options.set('seed', np.random.randint(2**31))

es = cma.CMAEvolutionStrategy([1,2], 0.01, options)

xs = np.array(es.ask())
print(xs)
es.tell(xs, [1,2,3,4])
print(es.result.xbest)
print(es.result.fbest)
print("="*10)
xs = np.array(es.ask())
print(xs)
es.tell(xs, [30,40,5,6])
print(es.result.xbest)
print(es.result.fbest)
print("="*10)
xs = np.array(es.ask())
print(xs)
es.tell(xs, [30,40,5,6])
print(es.result.xbest)
print(es.result.fbest)


# es.optimize(cma.ff.rosen)
# es.result_pretty()  # pretty print result

[[0.99191527 1.976241  ]
 [1.01057657 2.0156819 ]
 [1.01204235 2.01049521]
 [0.99050006 1.98470138]]
[0.99191527 1.976241  ]
1
[[1.00172657 1.99387972]
 [0.99505808 2.01205833]
 [0.9907601  2.00011739]
 [0.98450059 1.99546681]]
[0.99191527 1.976241  ]
1
[[0.98993253 1.97734342]
 [0.99260507 2.00333557]
 [0.98931992 1.99744514]
 [0.99271165 1.98640734]]
[0.99191527 1.976241  ]
1


In [4]:
# sys.path.append(f'../../../../Dataset/herg25oc1_real_data')
# from getdata import get_dataset, get_times

# # for i in range(5):
# # xs, ys = get_dataset(file_numbers=range(1, 3), multi=True, use_torch=True)

# # xs, ys = get_dataset(file_numbers=range(1, 31), window=10, step_size=5, multi=False, use_torch=False, get_raw=False)

# cell_to_remove=['A04', 'A15', 'E05', 'E22', 'G10', 'G22', 'H08', 'I10', 'I12', 'I16', 'L08', 'P15', 'P18' ] # M24
# cell_to_remove=[]

# xs, ys, cells = get_dataset(cell_to_remove=cell_to_remove, scale=1, multi=True, torch_tensor=False)  # <-- fast
# 

In [5]:
# VC_protocol = vc_protocols_for_myokit.leak_staircase()
VC_protocol = vc_protocols.leak_staircase()

print(f'The protocol is {VC_protocol.get_voltage_change_endpoints()[-1]} s')

The protocol is 15.4 s


In [6]:
if __name__=='__main__':
    freeze_support()
    
    # noise_removal = True
    
    
    savedir = './out'
    if not os.path.isdir(savedir):
        os.makedirs(savedir)
    
    data_dir = '../../../../Dataset/herg25oc1_real_data_nx1540' #-sweep2'  ####################################################### _nx1540
    qc_dir = '../qc'
    file_list = ['herg25oc1']
    temperatures = [25.0]
    useFilterCap = False
    
    # Control fitting seed --> OR DONT
    # fit_seed = np.random.randint(0, 2**30)
    fit_seed = 542811797
    print('Using seed: ', fit_seed)
    np.random.seed(fit_seed)
    
    noise_removal_li = [False]
    
    init_param_li = [ 'random' ]  # prior  |  real  |  predicted  |  random                           

    cell_li = [        
         'H04'
        
#         'A06', 'A09', 'A13', 'A16', 'A19', 'A22', 'B01', 'B04', 'B06', 'B10', 'B14', 'B17', 'B19', 'B21', 
#         'C02', 'C06', 'C09', 'C13', 'C15', 'C20', 'C22', 'D02', 'D04', 'D06', 'D09', 'D17', 'D19', 'D24', 
#         'E03', 'E06', 'E10', 'E15', 'E18', 'E21', 'F02','F10', 'F12', 'F16', 'F22', 
#         'G02', 'G07', 'G13', 'G21', 'H01', 'H04', 'H10', 'H12', 'H16', 'H19', 'H22', 
#         'I01', 'I04', 'I06', 'I12', 'I16', 'I20', 'I22', 'I24', 'J02', 'J04', 'J12', 'J14', 'J17', 'J22', 'J24', 
#         'K03', 'K05', 'K08', 'K11', 'K14', 'K17', 'K22', 'L01', 'L06', 'L16', 'L23', 
#         'M06', 'M10', 'M13', 'M19', 'M22', 'N03', 'N07', 'N10', 'N13', 'N18', 'N24', 
#         'O04', 'O09', 'O11', 'O15', 'O19', 'O24', 'P03', 'P09', 'P13', 'P15', 'P18', 'P21', 'P24'
    ]
    # file_list = os.listdir(data_dir)
    # param_list = [file for file in file_list if file.endswith("542811797.txt")]
    # for file in param_list:
    #     cell_li.append(file[24:27]) 
    # cell_li = cell_li[200:]
    # print(cell_li)

    sigma0 = None    
    
    
    # Set parameter transformation
#     transform_to_model_param = parametertransform.log_transform_to_model_param       # return np.exp(out)
#     transform_from_model_param = parametertransform.log_transform_from_model_param   # return np.log(out)
    transform_to_original = parametertransform.transform_to_original       # return np.exp(out)
    transform_to_norm = parametertransform.transform_to_norm    # return np.log(out) 
    
    nCPU = pints.ParallelEvaluator.cpu_count()
    print("The number of process :", nCPU)
    
    
        
    for i_file, (file_name, temperature) in enumerate(zip(file_list, temperatures)):
        # Load QC
        # selectedfile = '%s/selected-%s.txt' % (qc_dir, file_name)
        # selectedwell = []
        # with open(selectedfile, 'r') as f:
        #     for l in f:
        #         if not l.startswith('#'):
        #             selectedwell.append(l.split()[0])
    
        # Split each file_name as a separate output dir
        savefolder = file_name# + '-sweep2'
        if not os.path.isdir('%s/%s' % (savedir, savefolder)):
            os.makedirs('%s/%s' % (savedir, savefolder))
                
        for cell in cell_li:   
            for noise_removal in noise_removal_li:
                             


                # Load model
                model = m.Model('../../../mmt-model-files/kylie-2017-IKr.mmt',
                                protocol_def=protocol_def,
                                temperature=273.15 + temperature,  # K
                                transform=transform_to_original,
                                useFilterCap=useFilterCap)  # ignore capacitive spike

                print("Parameter inference for %s"%(cell))

                times = np.loadtxt(data_dir + '/' + file_name + '-staircaseramp-times.csv', delimiter=',', skiprows=1) # headers

                # Load data file names
                data_file_name = file_name + '-staircaseramp-' + cell + '.csv'

                # Save name
                saveas = data_file_name[:-4]
                # print(data_file_name)
                # print(saveas)
                if useFilterCap:
                    saveas += '-fcap'                

                # Load data
                data = np.loadtxt(data_dir + '/' + data_file_name, delimiter=',', skiprows=1) # headers

                noise_sigma = np.std(data[:500]) ## 1540 : data[:500]
                print('Estimated noise level: ', noise_sigma)
                print('sigma0: ', sigma0)

                # pre-processing for data #############################################
                if noise_removal:                
                    params_noise = {
                        'time' : times,
                        'vc' : VC_protocol,
                        'neighbor_radius' : 5,
                        'method' : 1
                    }
                    data = get_modified_current(params_noise, data)
                    print("Noise-removal method has been applied.")

                data = data[::50]
                times = times[::50]
                print(data.shape)
                print(times.shape)
                print("dt :", times[1])



                if useFilterCap:
                    # Apply capacitance filter to data
                    data = data * model.cap_filter(times)

                # Create Pints stuffs
                problem = pints.SingleOutputProblem(model, times, data)
                #loglikelihood = pints.KnownNoiseLogLikelihood(problem, noise_sigma)
                loglikelihood = pints.GaussianKnownSigmaLogLikelihood(problem, noise_sigma)
                logprior = LogPrior(transform_to_original, transform_to_norm)
                logposterior = pints.LogPosterior(loglikelihood, logprior)

                # MSE
                error_mse = pints.MeanSquaredError(problem) # MeanSquaredError RootMeanSquaredError  NormalisedRootMeanSquaredError

                # prior param  
                priorparams = np.asarray(prior_parameters['23.0']  )          
    #             priorparams = np.array([26086.585617, 0.276096, 64.486973, 0.019522, 49.343747, 107.503522, 8.068859, 6.581754, 31.552841])
                transformed_prior_params = transform_to_norm(priorparams)    # return np.log(out)

                # real parameters 
                params_file_name = file_name + '-staircaseramp-' + cell + '-solution-' + str(fit_seed) + '-dt001.txt'
                real_params = np.loadtxt(data_dir + '/' + params_file_name) # headers
                transformed_real_params = transform_to_norm(np.asarray(real_params)) 

                for init_param in init_param_li:
                    start_time = time.time()

                    eval_metric = None   
                    x0 = None
                    transformed_x0 = None

                    if init_param == 'prior':                               
                        eval_metric = logposterior                    
                        x0 = priorparams
                        transformed_x0 = transformed_prior_params

                    elif init_param == 'real':
                        eval_metric = logposterior  # error_mse    logposterior
                        x0 = real_params                    
                        transformed_x0 = transformed_real_params

                    elif init_param == 'predicted':   
                        params_file_name = cell + '.txt'                     
                        predicted_params = np.loadtxt('./Predicted_params/' + params_file_name) # headers                    

#                         predicted_params = np.array([29427.030528, 0.026970, 125.924017, 0.060602, 41.221571, 1461.519111, 39.967293, 50.841402, 21.259713])

                        eval_metric = logposterior  # error_mse    logposterior
                        x0 = predicted_params                    
                        transformed_x0 = transform_to_norm(np.asarray(predicted_params))    # return np.log(out)                    

                    elif init_param == 'random':    
                        eval_metric = logposterior                    
                        random.seed(None)
                        np.random.seed(None)

                        # transform_params = logprior.sample()                      
                        x0 = logprior.sample_without_inv_transform()
                        print(x0)
                        transformed_x0 = transform_to_norm(x0) 

                        # ikr_g = random.uniform(parameter_ranges[0][0], parameter_ranges[0][1])
                        # ikr_p1 = random.uniform(parameter_ranges[1][0], parameter_ranges[1][1])
                        # ikr_p2 = random.uniform(parameter_ranges[2][0], parameter_ranges[2][1])
                        # ikr_p3 = random.uniform(parameter_ranges[3][0], parameter_ranges[3][1])
                        # ikr_p4 = random.uniform(parameter_ranges[4][0], parameter_ranges[4][1])
                        # ikr_p5 = random.uniform(parameter_ranges[5][0], parameter_ranges[5][1])
                        # ikr_p6 = random.uniform(parameter_ranges[6][0], parameter_ranges[6][1])
                        # ikr_p7 = random.uniform(parameter_ranges[7][0], parameter_ranges[7][1])
                        # ikr_p8 = random.uniform(parameter_ranges[8][0], parameter_ranges[8][1])
                        # ikr_g = 0.5 * (parameter_ranges[0][0] + parameter_ranges[0][1])
                        # ikr_p1 = 0.5 * (parameter_ranges[1][0] + parameter_ranges[1][1])
                        # ikr_p2 = 0.5 * (parameter_ranges[2][0] + parameter_ranges[2][1])
                        # ikr_p3 = 0.5 * (parameter_ranges[3][0] + parameter_ranges[3][1])
                        # ikr_p4 = 0.5 * (parameter_ranges[4][0] + parameter_ranges[4][1])
                        # ikr_p5 = 0.5 * (parameter_ranges[5][0] + parameter_ranges[5][1])
                        # ikr_p6 = 0.5 * (parameter_ranges[6][0] + parameter_ranges[6][1])
                        # ikr_p7 = 0.5 * (parameter_ranges[7][0] + parameter_ranges[7][1])
                        # ikr_p8 = 0.5 * (parameter_ranges[8][0] + parameter_ranges[8][1])
                        # init_param = [ikr_g, ikr_p1, ikr_p2, ikr_p3, ikr_p4, ikr_p5, ikr_p6, ikr_p7, ikr_p8]
                        # transform_params = transform_from_model_param(np.asarray(random_parameters))    # return np.log(out)

                        np.random.seed(fit_seed)

                    else:
                        print("No initial parameters has been selected.")

                    mse = MSE(x0, real_params, axis=0)                       
                    mse_norm = MSE(transformed_x0, transformed_real_params, axis=0)
                    print("="*50)
                    print("Real parameters :", real_params)   
                    print("-----%s---------------"%(init_param))  
                    print('x0 :', x0)
                    print("transformed x0 :", transformed_x0)
                    print("MSE :", mse)
                    print("MSE for norm :", mse_norm)
                    print("="*50)

                    for _ in range(5):
                        assert(eval_metric(transformed_x0) ==\
                                eval_metric(transformed_x0))

    #                 try:
    #                     N = int(sys.argv[1])
    #                 except IndexError:
    #                     N = 4

                    # for i in range(N):

                    print('Starting point : ', x0)
                    print('Starting transformed point : ', transformed_x0)
                    print('Starting eval_metric : ', eval_metric(transformed_x0))
                    #opt = pints.Optimisation(logposterior, x0.T, method=pints.CMAES) # PSO  CMAES  GradientDescent  XNES  NelderMead   
                   # opt = pints.OptimisationController(error_mse, x0, method=pints.CMAES) # logposterior | error_mse

                    opt = OptimisationController(eval_metric, transformed_x0.T, sigma0=None, method=pints.CMAES)
                    population_size = os.cpu_count()
                    max_unchanged_iter = 100
                    unchanged_iter_threshold = 1e-11
                    opt._optimiser.set_population_size(population_size)  # nCPU
                    opt.set_max_iterations(None)            
                    opt.set_parallel(True) #72  36
                    opt.set_max_unchanged_iterations(iterations=max_unchanged_iter, threshold=unchanged_iter_threshold)  # iterations=200, threshold=1e-11                
    #                 opt.optimiser().set_local_global_balance(0.5)
                    #opt.optimiser().set_learning_rate(0.001)
                    #opt.set_population_size(population_size=80)
                    print("sigma0 :", opt._optimiser._sigma0)
                    print("max_iterations :", opt.max_iterations)
                    print("threshold :", opt.threshold)

                    # Run optimisation
                    transformed_params, params, logposteriors = [], [], []
                    try:
                        with np.errstate(all='ignore'):
                            # Tell numpy not to issue warnings
                            transformed_p, s = opt.run()                        
                            p = transform_to_original(transformed_p)
                            transformed_params.append(transformed_p)
                            params.append(p)                        
                            logposteriors.append(s)
                            print('Found solution:          True parameters:          Old parameters:' )
                            for k, x in enumerate(p):
                                print(pints.strfloat(x) + '    ' + \
                                        pints.strfloat(real_params[k]) + '    ' + \
                                            pints.strfloat(priorparams[k]))
                    except ValueError:    
                        continue
                        import traceback
                        traceback.print_exc()

                    mse = MSE(transformed_params[0], transformed_real_params, axis=0)

                    end_time = time.time()-start_time
                    print(transformed_params)
                    print(params)
                    print(logposteriors)                
                    print(opt.update_record)

                    savedir = 'Results-' + file_name + f"-staircaseramp-solution-CMAES-{fit_seed}-221022" ##################################################
                    if not os.path.isdir(savedir):
                        os.makedirs(savedir)
                    params_file_name = cell + '_%s_p%d_u%d_ui%s'%(init_param, population_size, max_unchanged_iter, str(unchanged_iter_threshold)[-2:]) + f"_nr{noise_removal}" + '.txt'

    #                 np.savetxt(savedir + params_file_name, params[0]) # headers
                    f = open(os.path.join(savedir, params_file_name), 'w')
                    f.write("Cell : %s\n"%(cell))
                    f.write("Initial : %s\n"%(init_param))       
                    f.write("population_size : %d\n"%(population_size))       
                    f.write("max_unchanged_iter : %d\n"%(max_unchanged_iter))                   
                    f.write("unchanged_iter_threshold : %e\n"%(unchanged_iter_threshold))    
                    if len(opt.update_record)!=0:
                        f.write("Epoch : %d  |  nUpdate : %d\n"%(opt.update_record[-1][0], opt.update_record[-1][1]))     
                    else:
                        f.write("Epoch : 0  |  nUpdate : 0\n")     
                    f.write("%e\n%e\n%e\n%e\n%e\n%e\n%e\n%e\n%e\n"%(params[0][0], params[0][1], params[0][2], params[0][3], params[0][4], params[0][5], params[0][6], params[0][7], params[0][8]))                
                    f.write("MSE_for_param : %e\n"%(mse))
                    f.write("best_score : %e\n"%(logposteriors[0]))
                    f.write("iter : %d\n"%opt.iterations())
                    f.write("Time : %f\n"%(opt.time()))                
                    f.close()          

                    print("--- %s seconds ---"%(end_time))                
                    print("MSE_for_param : %f"%(mse))
                    print("best_score : %f"%(logposteriors[0]))
                    print("iter : %d"%opt.iterations())
                    print("Time : %f"%(opt.time()))              
                    print("="*100)
                    print("="*100)
                    print("="*100)


                    if False:                  
                        # Load model
                        ion_model = m.Model('../../../mmt-model-files/kylie-2017-IKr.mmt',
                                        protocol_def=protocol_def,
                                        temperature=273.15 + temperature,  # K
                                        transform=None,
                                        useFilterCap=None)  # ignore capacitive spike
    #                     matplotlib.use('TkAgg')

                        real_sim_current = ion_model.simulate(np.array(real_params), times)    
                        predicted_current = ion_model.simulate(params[0], times)

                        fig, ax = plt.subplots(1,1, figsize=(15,6))
                        ax.plot(times, data, label='real')
                        # axs[0].set_xlim(0,2)
                        # axs[0].set_xlabel('Time [s]', fontsize=18)
                        ax.set_ylabel('Current [mV]', fontsize=14)
                        ax.grid(True)

                        ax.plot(times, real_sim_current, label='real_sim')
                        # axs[0].set_xlim(0,2)
                        # axs[0].set_xlabel('Time [s]', fontsize=18)
                        ax.set_ylabel('Current [mV]', fontsize=14)
                        ax.grid(True)

                        ax.plot(times, predicted_current, label='predicted')
                        # axs[1].set_xlim(0,2)
                        ax.set_xlabel('Time [s]', fontsize=14)
                        ax.set_ylabel('Current [pA]', fontsize=14)
                        ax.grid(True)

                        fig.tight_layout()
                        plt.legend()
                        plt.show()                

Using seed:  542811797
The number of process : 16
Parameter inference for H04
Estimated noise level:  10.236211953306926
sigma0:  None
(1540,)
(1540,)
dt : 0.01
[8.63186527e+03 1.18541290e-01 9.72706751e+01 1.20281881e-02
 1.34906511e+02 1.63469245e+03 7.41734135e+00 1.23176993e-01
 8.21960367e+01]
Real parameters : [5.18016172e+04 2.22426911e-01 7.69381758e+01 5.02190631e-02
 3.95658728e+01 8.98664840e+01 2.29513907e+01 6.49571050e+00
 3.20111434e+01]
-----random---------------
x0 : [8.63186527e+03 1.18541290e-01 9.72706751e+01 1.20281881e-02
 1.34906511e+02 1.63469245e+03 7.41734135e+00 1.23176993e-01
 8.21960367e+01]
transformed x0 : [0.01706714 0.30738696 0.25330886 0.20802002 0.70263792 0.72134361
 0.01931574 0.30905296 0.42810406]
MSE : 207336252.3100115
MSE for norm : 0.04188025251891233
Starting point :  [8.63186527e+03 1.18541290e-01 9.72706751e+01 1.20281881e-02
 1.34906511e+02 1.63469245e+03 7.41734135e+00 1.23176993e-01
 8.21960367e+01]
Starting transformed point :  [0.0170

In [7]:
print("천재")

천재
